In [6]:
# load model and predict
import os
import sys
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torch.autograd import Variable
import torchvision.transforms as transforms
import torchvision.models as models
import cv2 as cv
import pickle





In [2]:
# tree this /Users/Omar/Desktop/robocup/bag detection/runs/detect/train4
!tree /Users/Omar/Desktop/robocup/bag\ detection/runs/detect/train4

/Users/Omar/Desktop/robocup/bag detection/runs/detect/train4
├── args.yaml
├── events.out.tfevents.1703882106.Omars-MBP-6.11394.0
├── results.csv
├── train_batch0.jpg
├── train_batch1.jpg
├── train_batch2.jpg
└── weights
    ├── best.pt
    └── last.pt

2 directories, 8 files


In [62]:
import torch
from ultralytics import YOLO

# Path to the trained model file
model_path = "/Users/Omar/Desktop/robocup/bag detection/runs/detect/train4/weights/best.pt"

# Load the trained model
model = YOLO(model_path)


In [33]:
# print size of input model
print(model)

YOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        (act): SiLU(inplace=True)
      )
      (2): C2f(
        (cv1): Conv(
          (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1))
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(48, 32, kernel_size=(1, 1), stride=(1, 1))
          (act): SiLU(inplace=True)
        )
        (m): ModuleList(
          (0): Bottleneck(
            (cv1): Conv(
              (conv): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
              (act): SiLU(inplace=True)
            )
            (cv2): Conv(
              (conv): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
              (act): SiLU(inplace=True)
   

In [73]:
import cv2 as cv
import numpy as np

# Load and preprocess the image
img_path = '/Users/Omar/Desktop/robocup/bag detection/test.png'
img_path = '/Users/Omar/Desktop/robocup/test1.jpeg'
# img_path = '/Users/Omar/Desktop/robocup/bag detection/find-luggage-1/valid/images/bag_060_jpg.rf.33639af8da68b952a3586e43288ef37d.jpg'
img = cv.imread(img_path)
# img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
img = cv.resize(img, (1024, 704))  # Resize as per the model's input size
img = img.transpose(2, 0, 1)  # Change channel order
img = img[None, :, :, :]  # Add batch dimension
img = img.astype(np.float32) / 255.0  # Normalize to [0, 1]
img_tensor = torch.from_numpy(img)  # Convert to torch tensor


In [74]:
result = model.predict(img_tensor, conf=0.5)  # Get the results

print(result)  # Print the result



0: 704x1024 (no detections), 781.5ms
Speed: 0.9ms preprocess, 781.5ms inference, 12.3ms postprocess per image at shape (1, 3, 704, 1024)


[ultralytics.engine.results.Results object with attributes:

boxes: ultralytics.engine.results.Boxes object
keypoints: None
masks: None
names: {0: 'paper bag'}
orig_img: array([[[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       ...,

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 

In [75]:
bounding_boxes = result[0].boxes.xyxy.numpy()  # Get bounding boxes
name = result[0].names[0]  # Get the class name
print(bounding_boxes, name)  # Print the result

[] paper bag


In [76]:
# Load the original image again for drawing
orig_img = cv.imread(img_path)
# orig_img = cv.cvtColor(orig_img, cv.COLOR_BGR2RGB)

# No resizing needed for the original image
# print width and height of the image
print(orig_img.shape)


for box in bounding_boxes:
    # Coordinates - Convert to integers
    x1, y1, x2, y2 = int(box[0]), int(box[1]), int(box[2]), int(box[3])

    # Draw bounding box
    cv.rectangle(orig_img, (x1, y1), (x2, y2), (0, 255, 0), 2)

    # Put label with class name
    label = f'{name}'
    cv.putText(orig_img, label, (x1, y1 + 10), cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2)

# Display the image
cv.imshow('Detected Image', orig_img)
cv.waitKey(0)
cv.destroyAllWindows()


(225, 225, 3)


ultralytics.engine.results.Boxes object with attributes:

cls: tensor([0.])
conf: tensor([0.9325])
data: tensor([[  8.2898,   1.9543, 416.0000, 416.0000,   0.9325,   0.0000]])
id: None
is_track: False
orig_shape: (416, 416)
shape: torch.Size([1, 6])
xywh: tensor([[212.1449, 208.9772, 407.7102, 414.0457]])
xywhn: tensor([[0.5100, 0.5023, 0.9801, 0.9953]])
xyxy: tensor([[  8.2898,   1.9543, 416.0000, 416.0000]])
xyxyn: tensor([[0.0199, 0.0047, 1.0000, 1.0000]])
{0: 'paper bag'}
